## **Setup**
Click the Play Button. No need to expand. Do not touch this. 

## **FPGAs**

FPGA stands for Field Programmable Gate Array.
Just as it sounds, and FPGA is comprised of a large array of logic gates.
The useful thing about them is that these gates can be configured and reconfigured to do whatever you want.
This is very powerful becuase when is a design is implemented in hardware on an FPGA it is much faster that a similar disign would be running on a CPU.
Howevver, configuring thousands of these tiny gates by hand is nearly impossible. That is why we have Hardware Description Languages (HDL) like SystemVerilog. 
In HDL, we can describe the circuit that want to implement.
This HDL model has to go through a long process to produce a bitstream that can be downloaded to an FPGA.
This process can be devided into 3 main parts: synthesis, place and route, and bitstream generation. Each step requires complex software to effeciently implement your design on the specified FPGA.

<p align="left">
<img src="https://raw.githubusercontent.com/byuccl/digital_design_colab2/master/Tutorials/f4pga_overview/media/fpga_flow.png"
width="250" height="" style="display: block; margin: 0 auto" />
</p>



## **F4PGA Toolchain**

In these notebooks, we use the F4PGA toolchain to generate a bitstream from your SystemVerilog code. F4PGA stands for FOSS Flows For FPGA. FOSS stands for Free and Open Source Software. So F4PGA is simply a toolchain that combines opens source softwares to generate FPGA toolchains. They use different software for different steps of the process. For example, they use the [Surelog](https://github.com/chipsalliance/Surelog) parser to initial process your System Verilog code. Then they pass the design to [Yosys](https://yosyshq.net/yosys/) for synthesis, and on to [nextpnr](https://github.com/YosysHQ/nextpnr) for place and route. If you want to learn more about F4PGA, you can look visit their [website](https://f4pga.org/). At the end of the process you will have a bitstream that you can download to your FPGA.

### Installing the Toolchain

To install the toolchain in these notebooks, we use the commands from the following code blocks. It takes about 4 minutes for them to finish running. If you want to install it on your own machine, you can follow the [installation instructions](https://f4pga-examples.readthedocs.io/en/latest/getting.html#toolchain-installation) from the F4PGA website.

In [ ]:
!apt install -y git wget xz-utils

In [ ]:
%%bash
cd /content
git clone https://github.com/chipsalliance/f4pga-examples
cd /content/f4pga-examples
time wget https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh -O conda_installer.sh

In [ ]:
#Creates the Conda Environment
%%bash
cd /content/f4pga-examples
export F4PGA_INSTALL_DIR=~/opt/f4pga
export FPGA_FAM=xc7
time bash conda_installer.sh -u -b -p $F4PGA_INSTALL_DIR/$FPGA_FAM/conda;
time source "$F4PGA_INSTALL_DIR/$FPGA_FAM/conda/etc/profile.d/conda.sh";
time conda env create -f $FPGA_FAM/environment.yml

In [ ]:
#Download F4PGA Arch Defs
%%bash
export F4PGA_INSTALL_DIR=~/opt/f4pga
export FPGA_FAM=xc7
source "$F4PGA_INSTALL_DIR/$FPGA_FAM/conda/etc/profile.d/conda.sh";
conda activate xc7
mkdir -p $F4PGA_INSTALL_DIR/xc7/install

F4PGA_TIMESTAMP='20220920-124259'
F4PGA_HASH='007d1c1'
export F4PGA_PACKAGES='install-xc7 xc7a50t_test'

for PKG in $F4PGA_PACKAGES; do
  wget -qO- https://storage.googleapis.com/symbiflow-arch-defs/artifacts/prod/foss-fpga-tools/symbiflow-arch-defs/continuous/install/${F4PGA_TIMESTAMP}/symbiflow-arch-defs-${PKG}-${F4PGA_HASH}.tar.xz | tar -xJC $F4PGA_INSTALL_DIR/${FPGA_FAM}
done

### Using the toolchain

If you want to use the toolchain on your own there are just a few steps to get you set up. First you need to create a directory for the project. Put all of your source files here, as well as your XDC file. If these files are completed, you just need to add a makefile to your project directory. Here is an example makefile. The file should be called `Makefile`.

```
current_dir := ${CURDIR}
TARGET := basys3

# IMPORTANT: edit this next line if the name of your top level module is not 'top'
# You are not specifying a file name (there is no .sv).
# You specifying the name of the SystemVerilog that is the top level of your hierarchy.  
# Example: If your file is named myfile.sv but
#   inside that file you have `module myTopLevel(input logic clk, clr, ...`
# then you would specify `TOP := myTopLevel` on the next line.
TOP := top

# IMPORTANT: edit this next line to specify the name of your .xdc
# This IS a file name
XDC := ${current_dir}/file.xdc

# This next line will include all .v and .sv files in the current directory for compilation.
SOURCES := $(wildcard ${current_dir}/*.v ${current_dir}/*.sv)

# This actually does the compilation
include ${F4PGA_EXAMPLES_ROOT}/f4pga-examples/common/common.mk
```

The next thing you need to do is set some environmental variables and activate the conda environment. To this, we recommend creating a bash script to run every time you want to use the toolchain. Here is an example bash script. You can name it something like `f4activate.sh`.

```
export F4PGA_EXAMPLES_ROOT=~/                             # Path to f4pga-examples directory
export F4PGA_INSTALL_DIR=~/opt/f4pga                      # Path to f4pga directory which should be in an opt directory
export FPGA_FAM="xc7"                                     # Name of the f4pga fmaily you will be using
export PATH="$F4PGA_INSTALL_DIR/$FPGA_FAM/install/bin:$PATH";
source "$F4PGA_INSTALL_DIR/$FPGA_FAM/conda/etc/profile.d/conda.sh"
conda activate $FPGA_FAM
echo "Environment activated"
```

Now if you just execute this script with `source script.sh` and you should be able to run the toolchain. The conda environment name should be displayed in parenthesis befor your bash promt. The name of the environment should be the name of your FPGA family.

Make sure you are in your project directory, and then you can run the toolchain on your design. All you have to do is type `make` and it should run. We recommned using the Surelog parser since it has less bugs, and prints clearer error messages. To do this, just exted your make command to `SURELOG_CMD='-parse' make`. If you have your board on and plugged in, you can add the word `download` after `make` and it will automatically download your design to the board once the bitstream is generated. So your full command would be `SURELOG_CMD='-parse' make download`. You can also type `make download` anytime after your design has been compiled to download it to your board. 